In [1]:
import pandas as pd
import mysql.connector as connector

### Importing Data

In [2]:
df = pd.read_excel('Global Super Store dataset.xlsx')
df.head(3)

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,City,State,...,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,Shipping Cost,Order Priority
0,32298,CA-2012-124891,31-07-2012,31-07-2012,Same Day,RH-19495,Rick Hansen,Consumer,New York City,New York,...,TEC-AC-10003033,Technology,Accessories,Plantronics CS510 - Over-the-Head monaural Wir...,2309.650,7,0.0,762.1845,933.57,Critical
1,26341,IN-2013-77878,05-02-2013,07-02-2013,Second Class,JR-16210,Justin Ritter,Corporate,Wollongong,New South Wales,...,FUR-CH-10003950,Furniture,Chairs,"Novimex Executive Leather Armchair, Black",3709.395,9,0.1,-288.7650,923.63,Critical
2,25330,IN-2013-71249,17-10-2013,18-10-2013,First Class,CR-12730,Craig Reiter,Consumer,Brisbane,Queensland,...,TEC-PH-10004664,Technology,Phones,"Nokia Smart Phone, with Caller ID",5175.171,9,0.1,919.9710,915.49,Medium


### Preparing Data

Note that this is done along with Excel as well because it's easier to understand the sheet much better so that when writing scripts later to automate this process, it's easier

In [3]:
#Let's find missing values in each column
print('Missing Values in Each Column\n')
for i in df:
    print(f'{i} : {sum(df[i].isna())}')

Missing Values in Each Column

Row ID : 0
Order ID : 0
Order Date : 0
Ship Date : 0
Ship Mode : 0
Customer ID : 0
Customer Name : 0
Segment : 0
City : 0
State : 0
Country : 0
Postal Code : 41296
Market : 0
Region : 0
Product ID : 0
Category : 0
Sub-Category : 0
Product Name : 0
Sales : 0
Quantity : 0
Discount : 0
Profit : 0
Shipping Cost : 0
Order Priority : 0


In [4]:
#See how mnay unique values are there 
print("Unique values in,\n")
temp = ['City', 'State', 'Country',	'Market', 'Region']
for field in temp:
    print(f'{field} : {df[field].nunique()}')

Unique values in,

City : 3636
State : 1094
Country : 147
Market : 7
Region : 13


In [3]:
#Let's add a new calculated field as "Unit Price"
df['Unit Price']= (df['Sales'] / (1-df['Discount'])) / df['Quantity']
df.head(4)

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,City,State,...,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,Shipping Cost,Order Priority,Unit Price
0,32298,CA-2012-124891,31-07-2012,31-07-2012,Same Day,RH-19495,Rick Hansen,Consumer,New York City,New York,...,Technology,Accessories,Plantronics CS510 - Over-the-Head monaural Wir...,2309.650,7,0.0,762.1845,933.57,Critical,329.95
1,26341,IN-2013-77878,05-02-2013,07-02-2013,Second Class,JR-16210,Justin Ritter,Corporate,Wollongong,New South Wales,...,Furniture,Chairs,"Novimex Executive Leather Armchair, Black",3709.395,9,0.1,-288.7650,923.63,Critical,457.95
2,25330,IN-2013-71249,17-10-2013,18-10-2013,First Class,CR-12730,Craig Reiter,Consumer,Brisbane,Queensland,...,Technology,Phones,"Nokia Smart Phone, with Caller ID",5175.171,9,0.1,919.9710,915.49,Medium,638.91
3,13524,ES-2013-1579342,28-01-2013,30-01-2013,First Class,KM-16375,Katherine Murray,Home Office,Berlin,Berlin,...,Technology,Phones,"Motorola Smart Phone, Cordless",2892.510,5,0.1,-96.5400,910.16,Medium,642.78


### Extracting Table Entities from the Original DF

In [4]:
#Let's create dataframes that represents each table

customers_cols = ['Customer ID', 'Customer Name', 'Segment']
locations_cols = ['City', 'State', 'Country', 'Postal Code', 'Market', 'Region']
orders_cols = ['Order ID', 'Order Date', 'Sales', 'Quantity', 'Discount', 'Profit', 'Order Priority', 'Ship Date', 'Ship Mode', 'Shipping Cost', 'Customer ID']
products_cols = ['Product ID', 'Category', 'Sub-Category', 'Product Name', 'Unit Price']

In [5]:
table_names = ['Customers', 'Locations', 'Orders', 'Products']
vals = [customers_cols, locations_cols, orders_cols, products_cols]

In [6]:
tbs = {} #To store each table object temporarily before dropping duplicates

for i in range(4):
    temp_tb = df[vals[i]]
    tbs[table_names[i]] = temp_tb

In [7]:
tbs['Orders'].head(3)

,Order ID,Order Date,Sales,Quantity,Discount,Profit,Order Priority,Ship Date,Ship Mode,Shipping Cost,Customer ID
0,CA-2012-124891,31-07-2012,2309.650,7,0.0,762.1845,Critical,31-07-2012,Same Day,933.57,RH-19495
1,IN-2013-77878,05-02-2013,3709.395,9,0.1,-288.7650,Critical,07-02-2013,Second Class,923.63,JR-16210
2,IN-2013-71249,17-10-2013,5175.171,9,0.1,919.9710,Medium,18-10-2013,First Class,915.49,CR-12730


In [8]:
for key,val in tbs.items():
    print(f'Length of <{key}> table: {len(val)}')

Length of <Customers> table: 51290
Length of <Locations> table: 51290
Length of <Orders> table: 51290
Length of <Products> table: 51290


Let's get rid of the duplicates in each table

In [8]:
unique_tables = {} #To store new tables

#Let's remove duplicates for customers, orders, and products tables and add to unique tables disctionary (because they are straightforward)
unique_tables['Customer_tab'] = tbs['Customers'].drop_duplicates()
unique_tables['Orders_tab'] = tbs['Orders'].drop_duplicates()
unique_tables['Products_tab'] = tbs['Products'].drop_duplicates()

In [9]:
#Since Locations need a primary key, lets remove duplicates, add a primary key for unique values and store it in the dictionary object
loc_temp = tbs['Locations'].drop_duplicates()
#Adding the Location ID column
loc_temp.insert(0, 'Location ID', range(1, len(loc_temp) + 1))
unique_tables['locations_tab'] = loc_temp

In [10]:
for key, value in unique_tables.items():
    print(f'Length of table <{key}> : {len(value)}')

Length of table <Customer_tab> : 1590
Length of table <Orders_tab> : 51290
Length of table <Products_tab> : 17240
Length of table <locations_tab> : 3847


Orders is the same size here because every column together in the table, adds a unique record to the table.

Right now all the main tables are done. Now the challenging part. Creating the `Products-Orders` table & `Locations-Customers` table

Let's make the `Products-Orders` table. Here since some products share the same code, we are going to use the `Unit Price` of that product. This is why we added the `Unit Price` caculated field in the beginning of the code.

For example let's say Product ID is 004567. It has 2 products as "Leather Bat", "Mud guard". When this happens we can't distinguish between which product the customer has ordered when joining the orders table and products table for detailed view. Because of that we are using Unit price. So if the "Leather Bat" costs `$4.8` & "Mud guard" costs `$0.8`. We know what product actually the customer ordered for the Order's product ID of 004567 by calculating it's unit price.

In [11]:
#Let's create the products-orders table
orders_products_temp = df[['Order ID', 'Product ID', 'Unit Price']]
orders_products = orders_products_temp.drop_duplicates()
orders_products.tail(6)

,Order ID,Product ID,Unit Price
51284,IN-2014-57662,OFF-BI-10002424,12.90
51285,IN-2014-62366,OFF-FA-10000746,13.02
51286,US-2014-102288,OFF-AP-10002906,2.22
51287,US-2013-155768,OFF-EN-10001219,7.64
51288,MX-2012-140767,OFF-BI-10000806,6.72
51289,MX-2012-134460,OFF-PA-10004155,20.46


Now Let's create the `Locations-Customers` table. For this we need to create two temporary tables. 

* First we need a temporary location table replicate with an additional concat string to match along
* Second we need another temporary table for location table with customer ids without duplicates with the same concat string
* Then we need to check the location ID and add it to the second temporary table using Concat String
* Then we need to extract only the Location ID & Customer ID from the second temporary table & remove it's duplicates

In [12]:
#First table is the replicate of locations table with Concat String
temp_1 = unique_tables['locations_tab'].copy()
temp_1['Postal Code'] = temp_1['Postal Code'].astype(str) #Convert postal code to a string or an error will occur when adding concat string
temp_1['Concat String'] = temp_1[['City', 'State', 'Country', 'Market', 'Region', 'Postal Code']].agg('-'.join, axis=1)

In [13]:
#Notice we made a copy of locations table to produce the first temporary table needed to make the locations-customers table
#Check the differences in the columns
print(temp_1.columns)
print(unique_tables['locations_tab'].columns)

Index(['Location ID', 'City', 'State', 'Country', 'Postal Code', 'Market',
       'Region', 'Concat String'],
      dtype='object')
Index(['Location ID', 'City', 'State', 'Country', 'Postal Code', 'Market',
       'Region'],
      dtype='object')


In [14]:
temp_1.head(4)

,Location ID,City,State,Country,Postal Code,Market,Region,Concat String
0,1,New York City,New York,United States,10024.0,US,East,New York City-New York-United States-US-East-1...
1,2,Wollongong,New South Wales,Australia,nan,APAC,Oceania,Wollongong-New South Wales-Australia-APAC-Ocea...
2,3,Brisbane,Queensland,Australia,nan,APAC,Oceania,Brisbane-Queensland-Australia-APAC-Oceania-nan
3,4,Berlin,Berlin,Germany,nan,EU,Central,Berlin-Berlin-Germany-EU-Central-nan


In [15]:
#Next is the location table with customer ids
temp_2_temp = df[['City', 'State', 'Country', 'Postal Code', 'Market', 'Region', 'Customer ID']]
print(f'Before removing duplicates from second table: {len(temp_2_temp)}')
temp_2 = temp_2_temp.drop_duplicates() #When dropping duplicates, it needs to be saved to a new variable. Otherwise effects won't commit
print(f'After removing duplicates from second table: {len(temp_2)}')

Before removing duplicates from second table: 51290
After removing duplicates from second table: 25181


In [16]:
temp_2['Postal Code'] = temp_2['Postal Code'].astype(str)
temp_2['Concat String'] = temp_2[['City', 'State', 'Country', 'Market', 'Region', 'Postal Code']].agg('-'.join, axis=1)
temp_2.head(3)

C:\Users\USER\AppData\Local\Temp\ipykernel_7864\1592578394.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_2['Postal Code'] = temp_2['Postal Code'].astype(str)
C:\Users\USER\AppData\Local\Temp\ipykernel_7864\1592578394.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_2['Concat String'] = temp_2[['City', 'State', 'Country', 'Market', 'Region', 'Postal Code']].agg('-'.join, axis=1)


,City,State,Country,Postal Code,Market,Region,Customer ID,Concat String
0,New York City,New York,United States,10024.0,US,East,RH-19495,New York City-New York-United States-US-East-1...
1,Wollongong,New South Wales,Australia,nan,APAC,Oceania,JR-16210,Wollongong-New South Wales-Australia-APAC-Ocea...
2,Brisbane,Queensland,Australia,nan,APAC,Oceania,CR-12730,Brisbane-Queensland-Australia-APAC-Oceania-nan


Pandas Documentation About Merge: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html

In [17]:
#Let's match everything
result = pd.merge(temp_2, temp_1, on='Concat String', how='left')
result.head(3)

,City_x,State_x,Country_x,Postal Code_x,Market_x,Region_x,Customer ID,Concat String,Location ID,City_y,State_y,Country_y,Postal Code_y,Market_y,Region_y
0,New York City,New York,United States,10024.0,US,East,RH-19495,New York City-New York-United States-US-East-1...,1,New York City,New York,United States,10024.0,US,East
1,Wollongong,New South Wales,Australia,nan,APAC,Oceania,JR-16210,Wollongong-New South Wales-Australia-APAC-Ocea...,2,Wollongong,New South Wales,Australia,nan,APAC,Oceania
2,Brisbane,Queensland,Australia,nan,APAC,Oceania,CR-12730,Brisbane-Queensland-Australia-APAC-Oceania-nan,3,Brisbane,Queensland,Australia,nan,APAC,Oceania


In [18]:
#Let's extract only the required: Location ID & Customer ID
result_temp = result[['Customer ID','Location ID']]
customers_locations = result_temp.drop_duplicates()
(f'Length of <Customer-Locations> Table: {len(customers_locations)}')

'Length of <Customer-Locations> Table: 25181'

In [19]:
#Adding the Customer-Locations & Product-Order tables into "unique_tables" dictionary
unique_tables['customers_locations'] = customers_locations
unique_tables['orders_products'] = orders_products

When opened with Excel, I found that there are duplicates for the `Products` table:

Therefore, we need to remove duplicates specific column wise.

For products table, it's product_id which causes the problem (refer to the code below to get an idea from 1 instance)

In [20]:
#There are duplicate entries for the following product id in Products table, even after removing duplicates
unique_tables['Products_tab'][unique_tables['Products_tab']['Product ID'] == 'TEC-AC-10003033'] 

,Product ID,Category,Sub-Category,Product Name,Unit Price
0,TEC-AC-10003033,Technology,Accessories,Plantronics CS510 - Over-the-Head monaural Wir...,329.95
3021,TEC-AC-10003033,Technology,Accessories,Plantronics CS510 - Over-the-Head monaural Wir...,329.95


In [21]:
#Let's remove the duplicates
print(f'Before removing duplicates: {len(unique_tables['Products_tab'])}')
unique_tables['Products_tab'].drop_duplicates('Product ID', inplace=True)
print(f'After removing duplicates: {len(unique_tables['Products_tab'])}')

Before removing duplicates: 17240
After removing duplicates: 10292


C:\Users\USER\AppData\Local\Temp\ipykernel_7864\472613525.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_tables['Products_tab'].drop_duplicates('Product ID', inplace=True)


In [22]:
#Download every table in excel formats if needed
unique_tables['Customer_tab'].to_excel('customers.xlsx', index=False)
unique_tables['Orders_tab'].to_excel('orders.xlsx', index=False)
unique_tables['Products_tab'].to_excel('products.xlsx', index=False)
unique_tables['locations_tab'].to_excel('locations.xlsx', index=False)
unique_tables['customers_locations'].to_excel('customer_locations.xlsx', index=False)
unique_tables['orders_products'].to_excel('order_products.xlsx', index=False)

* I'm creating a function to import the above created tables to load and store in a dictionary.
* Doing this to save time without having go through runnin each line of code to create the tables from scratch

In [24]:
table_imports = {} #Change this name into unique_tables

def getTables():
    customers = pd.read_excel('Tables\customers.xlsx')
    locations = pd.read_excel('Tables\locations.xlsx')
    orders = pd.read_excel('Tables\orders.xlsx')
    products = pd.read_excel('Tables\products.xlsx')
    customer_location = pd.read_excel('Tables\customer_locations.xlsx')
    orders_products = pd.read_excel('Tables\order_products.xlsx')
    return [customers, locations, orders, products, customer_location, orders_products]

tables = getTables()

table_imports['Customer_tab']= tables[0]
table_imports['Orders_tab']= tables[2]
table_imports['Products_tab']= tables[3]
table_imports['locations_tab']= tables[1]
table_imports['customers_locations']= tables[-2]
table_imports['orders_products']= tables[-1]

<>:4: SyntaxWarning: invalid escape sequence '\c'
<>:5: SyntaxWarning: invalid escape sequence '\l'
<>:6: SyntaxWarning: invalid escape sequence '\o'
<>:7: SyntaxWarning: invalid escape sequence '\p'
<>:8: SyntaxWarning: invalid escape sequence '\c'
<>:9: SyntaxWarning: invalid escape sequence '\o'
<>:4: SyntaxWarning: invalid escape sequence '\c'
<>:5: SyntaxWarning: invalid escape sequence '\l'
<>:6: SyntaxWarning: invalid escape sequence '\o'
<>:7: SyntaxWarning: invalid escape sequence '\p'
<>:8: SyntaxWarning: invalid escape sequence '\c'
<>:9: SyntaxWarning: invalid escape sequence '\o'
C:\Users\USER\AppData\Local\Temp\ipykernel_7864\1574178824.py:4: SyntaxWarning: invalid escape sequence '\c'
  customers = pd.read_excel('Tables\customers.xlsx')
C:\Users\USER\AppData\Local\Temp\ipykernel_7864\1574178824.py:5: SyntaxWarning: invalid escape sequence '\l'
  locations = pd.read_excel('Tables\locations.xlsx')
C:\Users\USER\AppData\Local\Temp\ipykernel_7864\1574178824.py:6: SyntaxWarni

In [25]:
for key in table_imports.keys():
    print(key)

Customer_tab
Orders_tab
Products_tab
locations_tab
customers_locations
orders_products


In [26]:
for data in table_imports.values():
    print(len(data))

1590
51290
10292
3847
25181
51261


### Let's create databases, Populate them too

**Our database's ER diagram**

<div><img src="1.png" width="500"/></div>

In [27]:
#Importing db credentials
import json

def getCredentials():
    with open('db_credentials.json', 'r') as f:
        db_credentials = json.load(f)
        return db_credentials

In [28]:
#Let's create a connection with out database first
creds = getCredentials()
conn = connector.connect(user = creds['user'], password = creds['password'])
cur = conn.cursor()

In [57]:
#Creating the code needed to create the data tables
cur.execute('CREATE DATABASE GPS;')
cur.execute('SHOW DATABASES;') #Check if the database is created
cur.fetchall()

[('gps',),
 ('information_schema',),
 ('little_lemon',),
 ('lucky_shrub',),
 ('m&g_schema',),
 ('mangata_gallo',),
 ('mysql',),
 ('performance_schema',),
 ('sys',),
 ('temporary',),
 ('university',)]

In [29]:
#Since the database is created, let's use it
cur.execute('USE gps;')
conn.database #Checking which database is selected

'gps'

In [30]:
#Let's write a function to get the result as pandas df without having to write execute, fetchall methods
def getOutput(query):
    cur.execute(query)
    rows = cur.fetchall()
    cols = cur.column_names
    return pd.DataFrame(rows, columns=cols)

In [68]:
#Let's create the tables (SQL Statements to create tables)
customers = '''CREATE TABLE customers
(customerID VARCHAR(100) PRIMARY KEY NOT NULL, customerName VARCHAR(200) NOT NULL, segment VARCHAR(50) NOT NULL);'''

orders = '''CREATE TABLE orders
(orderID VARCHAR(100) PRIMARY KEY NOT NULL, orderDate DATE NOT  NULL, sales DECIMAL(6,2) NOT NULL, quantity INT NOT NULL, discount DECIMAL(2,1) NOT NULL, 
profit DECIMAL(6,2) NOT NULL, orderPriority VARCHAR(85) NOT NULL, shipDate DATE NOT NULL, shipMode VARCHAR(100) NOT NULL, shipCost DECIMAL(5,2) NOT NULL, 
customerID VARCHAR(100) NOT NULL, FOREIGN KEY (customerID) REFERENCES customers(customerID) ON DELETE CASCADE ON UPDATE CASCADE);'''

locations= '''CREATE TABLE locations 
(locationID INT PRIMARY KEY NOT NULL, city VARCHAR(100) NOT NULL, state VARCHAR(100), postalID INT, country VARCHAR(75)  NOT NULL, 
region VARCHAR(75) NOT NULL);'''

products= '''CREATE TABLE products
(productID VARCHAR(100) NOT NULL PRIMARY KEY, category VARCHAR(120) NOT NULL, subcategory VARCHAR(200) NOT NULL, productName VARCHAR(175) NOT NULL,
unitPrice DECIMAL(6,2) NOT NULL);'''

order_product= '''CREATE TABLE order_product
(orderID VARCHAR(100) NOT NULL, productID VARCHAR(100) NOT NULL, unitPrice DECIMAL(6,2), PRIMARY KEY(orderID, productID), 
FOREIGN KEY(orderID) REFERENCES orders(orderID) ON DELETE CASCADE ON UPDATE CASCADE,
FOREIGN KEY(productID) REFERENCES products(productID) ON DELETE CASCADE ON UPDATE CASCADE);'''

customer_location= '''CREATE TABLE customer_location
(customerID VARCHAR(100) NOT NULL, locationID INT NOT NULL, PRIMARY KEY(customerID, locationID),
FOREIGN KEY(customerID) REFERENCES customers(customerID) ON DELETE CASCADE ON UPDATE CASCADE,
FOREIGN KEY(locationID) REFERENCES locations(locationID) ON DELETE CASCADE ON UPDATE CASCADE);'''

In [69]:
tables = [customers, orders, locations, products, order_product, customer_location]

#Let's create the data tables
for table in tables:
    cur.execute(table)

In [70]:
getOutput('SHOW TABLES;')

,Tables_in_gps
0,customer_location
1,customers
2,locations
3,order_product
4,orders
5,products


In [33]:
getOutput('SHOW COLUMNS FROM products;')

,Field,Type,Null,Key,Default,Extra
0,productID,varchar(100),NO,PRI,None,
1,category,varchar(120),NO,,None,
2,subcategory,varchar(200),NO,,None,
3,productName,varchar(175),NO,,None,
4,unitPrice,"decimal(6,2)",NO,,None,


### Populating the tables

In [72]:
#Let's write the insert queries for each table
pop_customers = '''INSERT INTO customers(customerID, customerName, segment) VALUES(%s, %s, %s);'''
pop_locations = '''INSERT INTO locations(locationID, city, state, postalID, country, region) VALUES(%s, %s, %s, %s, %s, %s);'''
pop_products = '''INSERT INTO products(productID, category, subcategory, productName, unitPrice) VALUES(%s, %s, %s, %s, %s);'''
pop_orders = '''INSERT INTO orders(orderID, orderDate, sales, quantity, discount, profit, orderPriority, shipDate, shipMode , shipCost, customerID) 
VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);'''
pop_customer_location = '''INSERT INTO customer_location(customerID, locationID) VALUES(%s, %s);'''
pop_order_product = '''INSERT INTO order_product(orderID, productID, unitPrice) VALUES(%s, %s, %s);'''

In [58]:
populate_queries = [pop_customers, pop_orders, pop_products, pop_locations, pop_customer_location, pop_order_product]

In [52]:
#Addin each table into a list
import_from = [i for i in unique_tables.values()]

In [82]:
#populating in manual way (Automatic method didn't work yet)
data = []
for iter in range(len(import_from)):
    temp = []
    for index, row in import_from[iter].iterrows():
            temp.append(tuple(row))
    data.append(temp)       

In [84]:
for i in data:
    print(len(i))

1590
51290
10292
3847
25181
51261


In [96]:
for data_item, query in zip(data, populate_queries):
    cur.executemany(query, data_item)

IntegrityError: 1062 (23000): Duplicate entry 'RH-19495' for key 'customers.PRIMARY'

In [87]:
for i in range(len(data)):
    cur.executemany(data[i], populate_queries[i])

ProgrammingError: Parameters for query must be list or tuple.

In [93]:
data[4]

[('RH-19495', 1),
 ('JR-16210', 2),
 ('CR-12730', 3),
 ('KM-16375', 4),
 ('RH-9495', 5),
 ('JM-15655', 6),
 ('TS-21340', 7),
 ('MB-18085', 8),
 ('JW-15220', 9),
 ('JH-15985', 10),
 ('GM-14695', 11),
 ('AJ-10780', 12),
 ('MM-7260', 13),
 ('VF-21715', 14),
 ('PF-19120', 15),
 ('BP-11185', 16),
 ('TB-21175', 17),
 ('PJ-18835', 18),
 ('JS-15685', 19),
 ('RH-9555', 20),
 ('AB-600', 21),
 ('SA-20830', 22),
 ('JK-15325', 23),
 ('LB-16795', 24),
 ('NP-18325', 25),
 ('VD-21670', 26),
 ('PB-19210', 27),
 ('EB-14110', 28),
 ('KF-16285', 29),
 ('BP-11230', 30),
 ('RR-19525', 31),
 ('BS-11365', 32),
 ('JE-15745', 33),
 ('DP-13105', 34),
 ('NP-18700', 35),
 ('AH-30', 36),
 ('DM-13015', 37),
 ('GT-14635', 38),
 ('PO-18865', 39),
 ('DL-12865', 3),
 ('JB-16000', 40),
 ('BF-11005', 41),
 ('VG-21805', 42),
 ('GT-14710', 43),
 ('ZC-21910', 4),
 ('SC-20095', 44),
 ('EB-13840', 45),
 ('AP-10915', 46),
 ('SW-20275', 47),
 ('JH-15820', 48),
 ('LC-16885', 37),
 ('TG-11640', 49),
 ('HR-14830', 50),
 ('DG-3300',